# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np

In [ ]:
#| export
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

In [ ]:
# Test Variable
data = np.array(1.0)
x = Variable(data)
assert x.data == np.array(1.0), print(x.data)

x.data = np.array(2.0)
assert x.data == np.array(2.0), print(x.data)

In [ ]:
#| export
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, in_data):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

In [ ]:
#| export
class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

In [ ]:
# Test Square
x = Variable(np.array(10))
f = Square()
y = f(x)
assert type(y)==Variable
assert y.data==100

In [ ]:
#| export
class Exp(Function):
    def forward(self, x):
        return np.exp(x)

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

In [ ]:
# Test Exp
x = Variable(np.array(2.))
f = Exp()
y = f(x)
assert np.allclose(y.data, 7.3890561)

In [ ]:
# Test concatenate
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
assert np.allclose(y.data, 1.648721270700128)

In [ ]:
#| export
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

In [ ]:
# test numerical_diff
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)

assert np.allclose(dy, 4)

In [ ]:
# concatenate test with numerical_diff
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
assert np.allclose(dy, 3.2974426293330694)

In [ ]:
# Test backward
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)

assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

assert np.allclose(x.grad, 3.297442541400256)

AssertionError: 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()